<a href="https://colab.research.google.com/github/phaethonp/we-ai/blob/main/IFC2JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ifcopenshell
!pip install pandas-profiling


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## IFC TO JSON ##


In [ ]:
import os
import json
import ifcopenshell
import os
import uuid
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.guid as guid
from datetime import datetime
from ifcopenshell.entity_instance import entity_instance
import pandas as pd

In [ ]:
class IFC2JSON:
    """Base class for all IFC SPF to IFC.JSON writers
    """

    VERSION = '0.0.1'

    DIMENSIONALEXPONENTS = {
        'METRE': (1, 0, 0, 0, 0, 0, 0),
        'SQUARE_METRE': (2, 0, 0, 0, 0, 0, 0),
        'CUBIC_METRE': (3, 0, 0, 0, 0, 0, 0),
        'GRAM': (0, 1, 0, 0, 0, 0, 0),
        'SECOND': (0, 0, 1, 0, 0, 0, 0),
        'AMPERE': (0, 0, 0, 1, 0, 0, 0),
        'KELVIN': (0, 0, 0, 0, 1, 0, 0),
        'MOLE': (0, 0, 0, 0, 0, 1, 0),
        'CANDELA': (0, 0, 0, 0, 0, 0, 1),
        'RADIAN': (0, 0, 0, 0, 0, 0, 0),
        'STERADIAN': (0, 0, 0, 0, 0, 0, 0),
        'HERTZ': (0, 0, -1, 0, 0, 0, 0),
        'NEWTON': (1, 1, -2, 0, 0, 0, 0),
        'PASCAL': (-1, 1, -2, 0, 0, 0, 0),
        'JOULE': (2, 1, -2, 0, 0, 0, 0),
        'WATT': (2, 1, -3, 0, 0, 0, 0),
        'COULOMB': (0, 0, 1, 1, 0, 0, 0),
        'VOLT': (2, 1, -3, -1, 0, 0, 0),
        'FARAD': (-2, -1, 4, 2, 0, 0, 0),
        'OHM': (2, 1, -3, -2, 0, 0, 0),
        'SIEMENS': (-2, -1, 3, 2, 0, 0, 0),
        'WEBER': (2, 1, -2, -1, 0, 0, 0),
        'TESLA': (0, 1, -2, -1, 0, 0, 0),
        'HENRY': (2, 1, -2, -2, 0, 0, 0),
        'DEGREE_CELSIUS': (0, 0, 0, 0, 1, 0, 0),
        'LUMEN': (0, 0, 0, 0, 0, 0, 1),
        'LUX': (-2, 0, 0, 0, 0, 0, 1),
        'BECQUEREL': (0, 0, -1, 0, 0, 0, 0),
        'GRAY': (2, 0, -2, 0, 0, 0, 0),
        'SIEVERT': (2, 0, -2, 0, 0, 0, 0),
        'OTHERWISE': (0, 0, 0, 0, 0, 0, 0)
    }

    def toLowerCamelcase(self, string):
        """Convert string from upper to lower camelCase"""

        return string[0].lower() + string[1:]

    def getDimensionsForSiUnit(self, entity):
        dimensions = {
            'type': 'IfcDimensionalExponents'
        }
        if entity.Name in self.DIMENSIONALEXPONENTS:
            dimExps = self.DIMENSIONALEXPONENTS[entity.Name]
            if dimExps[0] != 0:
                dimensions['LengthExponent'] = dimExps[0]
            if dimExps[1] != 0:
                dimensions['MassExponent'] = dimExps[1]
            if dimExps[2] != 0:
                dimensions['TimeExponent'] = dimExps[2]
            if dimExps[3] != 0:
                dimensions['ElectricCurrentExponent'] = dimExps[3]
            if dimExps[4] != 0:
                dimensions['ThermodynamicTemperatureExponent'] = dimExps[4]
            if dimExps[5] != 0:
                dimensions['AmountOfSubstanceExponent'] = dimExps[5]
            if dimExps[6] != 0:
                dimensions['LuminousIntensityExponent'] = dimExps[6]

        return dimensions

    def getAttributeValue(self, value):
        """Recursive method that walks through all nested objects of an attribute
        and returns a IFC.JSON-4 model structure

        Parameters:
        value

        Returns:
        attribute data converted to IFC.JSON-4 model structure

        """
        if value == None or value == '':
            jsonValue = None
        elif isinstance(value, ifcopenshell.entity_instance):
            entity = value
            entityAttributes = entity.__dict__

            # Remove empty properties
            if entity.is_a('IfcProperty'):
                if not self.EMPTY_PROPERTIES:
                    if self.empty_property(entity):
                        return None

            # Add unit dimensions https://standards.buildingsmart.org/IFC/DEV/IFC4_2/FINAL/HTML/schema/ifcmeasureresource/lexical/ifcdimensionsforsiunit.htm
            if entity.is_a('IfcSIUnit'):
                entityAttributes['dimensions'] = self.getDimensionsForSiUnit(
                    entity)

            # All objects with a GlobalId must be referenced, all others nested
            if entity.id() in self.rootObjects:
                entityAttributes["GlobalId"] = self.rootObjects[entity.id()]
                return self.createReferenceObject(entityAttributes, self.COMPACT)
            else:
                if 'GlobalId' in entityAttributes:
                    entityAttributes["GlobalId"] = guid.split(
                        guid.expand(entity.GlobalId))[1:-1]

            return self.createFullObject(entityAttributes)
        elif isinstance(value, tuple):
            jsonValue = tuple(x for x in map(
                self.getAttributeValue, value) if x is not None)
        else:
            jsonValue = value
        return jsonValue

    def empty_property(self, entity):

        # IfcPropertySingleValue
        if hasattr(entity, 'NominalValue'):
            if not entity.NominalValue:
                return True
            elif entity.NominalValue:
                value = entity.NominalValue.wrappedValue
                if (not value and value is not False) or (value == ''):
                    return True

        # IfcPropertyEnumeratedValue
        elif hasattr(entity, 'EnumerationValues'):
            if not entity.EnumerationValues:
                return True

        # IfcPropertyBoundedValue
        elif hasattr(entity, 'UpperBoundValue'):
            if not entity.UpperBoundValue and not entity.LowerBoundValue:
                return True

        # IfcPropertyTableValue
        elif hasattr(entity, 'DefiningValues'):
            if not entity.DefiningValues and not entity.DefinedValues:
                return True


In [ ]:
# The IFC2JSON5a class is a subclass of IFC2JSON that provides additional functionality for converting IfcProducts to OBJ meshes in the IFC.JSON-5a format.
# It introduces the toObj method, which takes an ifcopenshell.ifcProduct instance and converts it to an OBJ string representing the geometry of the product. It uses the ifcopenshell.geom.create_shape function to create a shape representation of the product and then extracts the vertices and faces from the shape to generate the OBJ string.
# The toObj method is called for each product with a representation, and the resulting OBJ string is added to the representations dictionary, which will be included in the final JSON output.
# The spf2Json method is extended to handle the conversion of representations to OBJ and include them in the JSON output.
# The createFullObject method is modified to exclude attributes that are not part of the IFC.JSON-5a format, as specified in the INVALIDATTRIBUTES attribute of the class.
# The createFullObject method is also modified to flatten the object hierarchy by removing intermediate relationship objects specified in the SIMPLIFICATIONS attribute of the class.
# Overall, the IFC2JSON5a class provides the capability to convert IfcProducts to OBJ meshes and include them in the IFC.JSON-5a output.

class IFC2JSON5a(IFC2JSON):
    SCHEMA_VERSION = '0.0.1'

    # Attributes that are not part of IFC.JSON5a
    INVALIDATTRIBUTES = {
        'OwnerHistory',
        'RepresentationContexts',
        'ContextOfItems',
        'ObjectPlacement',
        'RepresentationMaps'
    }

    # Attributes for which the intermediate relationship object is removed
    SIMPLIFICATIONS = {
        # IfcRelAggregates
        'IsDecomposedBy':       ['relatedObjects'],
        'Decomposes':           ['relatingObject'],
        # IfcRelContainedInSpatialStructure
        'ContainsElements':     ['relatedElements'],
        'ContainedInStructure': ['relatingStructure'],
        # IfcRelDefinesByProperties
        'IsDefinedBy':          ['relatingPropertyDefinition', 'relatingType'],
        # IfcRelAssociatesMaterial
        'HasAssociations':      ['relatingMaterial'],
        # IfcRelFillsElement
        'HasFillings':          ['relatedBuildingElement'],
        'FillsVoids':           ['relatingOpeningElement'],
        # IfcRelVoidsElement
        'HasOpenings':          ['relatedOpeningElement'],
        'VoidsElements':        ['relatingBuildingElement'],
        # IfcRelDefinesByType
        'ObjectTypeOf':         ['relatedObjects'],
        'IsTypedBy':            ['relatingType'],
        # IfcRelConnectsPathElements (!) This skips all IfcRelConnectsPathElements properties
        'ConnectedTo':          ['relatedElement'],
        'ConnectedFrom':        ['relatingElement'],
        # IfcRelSpaceBoundary (!) This skips all spaceboundary properties like for example geometry
        'BoundedBy':            ['relatedBuildingElement'],
        'ProvidesBoundaries':   ['relatingSpace']
    }

    settings = ifcopenshell.geom.settings()
    settings.USE_PYTHON_OPENCASCADE = True
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.EXCLUDE_SOLIDS_AND_SURFACES, False)

    def __init__(self, ifcModel,
                 COMPACT=False,
                 EMPTY_PROPERTIES=False):
        """IFC SPF to IFC.JSON-5a writer

        parameters:
        ifcModel: IFC filePath or ifcopenshell model instance
        COMPACT (boolean): if True then pretty print is turned off and references are created without informative "type" property

        """
        if isinstance(ifcModel, ifcopenshell.file):
            self.ifcModel = ifcModel
        else:
            self.ifcModel = ifcopenshell.open(ifcModel)
        self.COMPACT = COMPACT
        self.EMPTY_PROPERTIES = EMPTY_PROPERTIES

        # Dictionary referencing all objects with a GlobalId that are already created
        self.rootObjects = {}

        # Representations are kept seperate to be added to the end of the list
        self.representations = {}

    def spf2Json(self):
        """
        Create json dictionary structure for all attributes of the objects in the root list
        (?) also including inverse attributes
        (?) Check every IFC object to see if it is used multiple times

        Returns:
        dict: IFC.JSON-5a model structure

        """

        jsonObjects = []

        for entity in self.ifcModel.by_type('IfcObjectDefinition'):
            self.rootObjects[entity.id()] = guid.split(
                guid.expand(entity.GlobalId))[1:-1]

        for key in self.rootObjects:
            entity = self.ifcModel.by_id(key)
            entityAttributes = entity.__dict__
            entityType = entityAttributes['type']
            if not entityType in ['IfcGeometricRepresentationContext', 'IfcOwnerHistory']:
                for attr in entity.wrapped_data.get_inverse_attribute_names():
                    inverseAttribute = getattr(entity, attr)
                    entityAttributes[attr] = self.getAttributeValue(
                        inverseAttribute)
            entityAttributes["GlobalId"] = self.rootObjects[entity.id()]

            # Convert representations to OBJ
            if 'Representation' in entityAttributes:
                obj = self.toObj(entity)

                if obj:
                    id = guid.split(guid.expand(guid.new()))[1:-1]
                    ref = {}
                    if not self.COMPACT:
                        ref['type'] = "shapeRepresentation"
                    ref['ref'] = id
                    entityAttributes['representations'] = [ref]
                    self.representations[id] = {
                        "type": "shapeRepresentation",
                        "globalId": id,
                        "representationIdentifier": "Body",
                        "representationType": "OBJ",
                        "items": [
                            obj
                        ]
                    }

                # (!) delete original representation, even if OBJ generation fails
                del entityAttributes['Representation']

            jsonObjects.append(self.createFullObject(entityAttributes))

        jsonObjects = jsonObjects + list(self.representations.values())

        return {
            'type': 'IFC.JSON-5a',
            'version': self.SCHEMA_VERSION,
            'schemaIdentifier': self.ifcModel.wrapped_data.schema,
            'originatingSystem': 'IFC2JSON_python Version ' + self.VERSION,
            'preprocessorVersion': 'IfcOpenShell ' + ifcopenshell.version,
            'timeStamp': datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            'data': jsonObjects
        }

    def createFullObject(self, entityAttributes):
        """Returns complete IFC.JSON-5a object

        Parameters:
        entityAttributes (dict): Dictionary of IFC object data

        Returns:
        dict: containing complete IFC.JSON-5a object

        """
        fullObject = {}

        for attr in entityAttributes:

            # Line numbers are not part of IFC JSON
            if attr == 'id':
                continue

            # Skip all IFC entities that are not part of IFC.JSON5a
            if attr in self.INVALIDATTRIBUTES:
                continue

            # Flatten object hierarchy through removing intermediate relationship objects
            if attr in self.SIMPLIFICATIONS:
                for relObject in entityAttributes[attr]:
                    for attrName in self.SIMPLIFICATIONS[attr]:

                        # In case of propertysets, further simplification through removing intermediate PropertySets
                        if attr == 'IsDefinedBy':
                            if relObject['type'] == 'RelDefinesByProperties':
                                if relObject['relatingPropertyDefinition']:
                                    relatingPropertyDefinition = relObject['relatingPropertyDefinition']
                                    if 'hasProperties' in relatingPropertyDefinition:
                                        for property in relatingPropertyDefinition['hasProperties']:
                                            try:
                                                fullObject[property['name']
                                                           ] = property['nominalValue']['value']
                                            except Exception as e:
                                                print(str(e))
                                continue
                            else:
                                print(relObject['type'])
                        if attrName in relObject:
                            entityAttributes[attr] = relObject[attrName]

            attrKey = self.toLowerCamelcase(attr)

            # Replace wrappedvalue key names to value
            if attrKey == 'wrappedValue':
                attrKey = 'value'

            jsonValue = self.getAttributeValue(entityAttributes[attr])
            if jsonValue is not None:

                # Entity names must be stripped of Ifc prefix
                if attr == 'type':
                    jsonValue = jsonValue[3:]

                fullObject[attrKey] = jsonValue
        return fullObject

    def createReferenceObject(self, entityAttributes, COMPACT=False):
        """Returns object reference

        Parameters:
        entityAttributes (dict): Dictionary of IFC object data
        COMPACT (boolean): verbose or non verbose IFC.JSON-5a output

        Returns:
        dict: object containing reference to another object

        """
        ref = {}
        if not COMPACT:

            # Entity names must be stripped of Ifc prefix
            ref['type'] = entityAttributes['type'][3:]
        ref['ref'] = entityAttributes['GlobalId']
        return ref

    def toObj(self, product):
        """Convert IfcProduct to OBJ mesh

        parameters:
        product: ifcopenshell ifcProduct instance

        Returns:
        string: OBJ string
        """

        if product.Representation:
            try:
                shape = ifcopenshell.geom.create_shape(self.settings, product)

                verts = shape.geometry.verts
                vertsList = [' '.join(map(str, verts[x:x+3]))
                             for x in range(0, len(verts), 3)]
                vertString = 'v ' + '\nv '.join(vertsList) + '\n'

                faces = shape.geometry.faces
                facesList = [' '.join(map(str, faces[x:x+3]))
                             for x in range(0, len(faces), 3)]
                faceString = 'f ' + '\nf '.join(map(str, facesList)) + '\n'

                return vertString + faceString
            except Exception as e:
                print(str(e) + ': Unable to generate OBJ data for ' +
                      str(product))
                return None

In [ ]:
class IFC2JSON4(IFC2JSON):
    SCHEMA_VERSION = '0.0.1'

    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, False)

    def __init__(self,
                 ifcModel,
                 COMPACT=False,
                 INCLUDE_INVERSE=False,
                 EMPTY_PROPERTIES=False,
                 NO_OWNERHISTORY=False,
                 GEOMETRY=True):
        """IFC SPF to IFC.JSON-4 writer

        parameters:
        ifcModel: IFC filePath or ifcopenshell model instance
        COMPACT (boolean): if True then pretty print is turned off and references are created without informative "type" property
        INCLUDE_INVERSE (boolean): if True then inverse relationships will be explicitly added to entities

        """

        if isinstance(ifcModel, ifcopenshell.file):
            self.ifcModel = ifcModel
        else:
            self.ifcModel = ifcopenshell.open(ifcModel)
        # input(dir(self.ifcModel.wrapped_data.header))
        # input(self.ifcModel.wrapped_data.header)
        # print(dir(self.ifcModel.wrapped_data.header.file_description))
        # if self.ifcModel.wrapped_data.header.file_description.this:
        #     print(self.ifcModel.wrapped_data.header.file_description[0])
        # input()
        self.COMPACT = COMPACT
        self.INCLUDE_INVERSE = INCLUDE_INVERSE
        self.EMPTY_PROPERTIES = EMPTY_PROPERTIES
        self.rootObjects = {}

        if NO_OWNERHISTORY:
            self.remove_ownerhistory()

        # adjust GEOMETRY type
        if GEOMETRY == 'tessellate':
            self.tessellate()
        elif GEOMETRY == False:
            self.remove_geometry()

    def spf2Json(self):
        """
        Create json dictionary structure for all attributes of the objects in the root list
        also including inverse attributes (except for IfcGeometricRepresentationContext and IfcOwnerHistory types)
        # (?) Check every IFC object to see if it is used multiple times

        Returns:
        dict: IFC.JSON-4 model structure

        """

        jsonObjects = []
        relationships = []

        # Collect all entity types that already have a GlobalId
        for entity in self.ifcModel.by_type('IfcRoot'):
            if entity.is_a('IfcRelationship'):
                relationships.append(entity)
            else:
                self.rootObjects[entity.id()] = guid.split(
                    guid.expand(entity.GlobalId))[1:-1]

        # seperately collect all entity types where a GlobalId needs to be added
        # for entity in self.ifcModel.by_type('IfcMaterialDefinition'):
        #     self.rootObjects[entity.id()] = str(uuid.uuid4())
        for entity in self.ifcModel.by_type('IfcShapeRepresentation'):
            self.rootObjects[entity.id()] = str(uuid.uuid4())
        for entity in self.ifcModel.by_type('IfcOwnerHistory'):
            self.rootObjects[entity.id()] = str(uuid.uuid4())
        for entity in self.ifcModel.by_type('IfcGeometricRepresentationContext'):
            self.rootObjects[entity.id()] = str(uuid.uuid4())

        # Seperately add all IfcRelationship entities so they appear at the end of the list
        for entity in relationships:
            self.rootObjects[entity.id()] = guid.split(
                guid.expand(entity.GlobalId))[1:-1]

        for key in self.rootObjects:
            entity = self.ifcModel.by_id(key)
            entityAttributes = entity.__dict__
            entityType = entityAttributes['type']
            if not entityType == 'IfcOwnerHistory':
                if not self.INCLUDE_INVERSE:
                    for attr in entity.wrapped_data.get_inverse_attribute_names():
                        inverseAttribute = getattr(entity, attr)
                        attrValue = self.getAttributeValue(inverseAttribute)
                        if not attrValue and attrValue is not False:
                            continue
                        else:
                            entityAttributes[attr] = attrValue

            entityAttributes["GlobalId"] = self.rootObjects[entity.id()]
            jsonObjects.append(self.createFullObject(entityAttributes))

        return {
            'type': 'IFC.JSON',
            'version': self.SCHEMA_VERSION,
            'schemaIdentifier': self.ifcModel.wrapped_data.schema,
            'originatingSystem': 'IFC2JSON_python Version ' + self.VERSION,
            'preprocessorVersion': 'IfcOpenShell ' + ifcopenshell.version,
            'timeStamp': datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            'data': jsonObjects
        }

    def createFullObject(self, entityAttributes):
        """Returns complete IFC.JSON-4 object

        Parameters:
        entityAttributes (dict): Dictionary of IFC object data

        Returns:
        dict: containing complete IFC.JSON-4 object

        """
        fullObject = {}

        for attr in entityAttributes:

            # Line numbers are not part of IFC JSON
            if attr == 'id':
                continue

            attrKey = self.toLowerCamelcase(attr)

            # Replace wrappedvalue key names to value
            if attrKey == 'wrappedValue':
                attrKey = 'value'

            jsonValue = self.getAttributeValue(entityAttributes[attr])
            if jsonValue is not None:
                fullObject[attrKey] = jsonValue
        return fullObject

    def createReferenceObject(self, entityAttributes, COMPACT=False):
        """Returns object reference

        Parameters:
        entityAttributes (dict): Dictionary of IFC object data
        COMPACT (boolean): verbose or non verbose IFC.JSON-4 output

        Returns:
        dict: object containing reference to another object

        """
        ref = {}
        if not COMPACT:
            ref['type'] = entityAttributes['type']
        ref['ref'] = entityAttributes['GlobalId']
        return ref

    def tessellate(self):
        """Converts all IfcProduct representations to IfcTriangulatedFaceSet
        """
        for product in self.ifcModel.by_type('IfcProduct'):
            if product.Representation:
                try:
                    representation = product.Representation
                    old_shapes = representation.Representations
                    context = old_shapes[0].ContextOfItems

                    tessellated_shape = ifcopenshell.geom.create_shape(
                        self.settings, product)

                    verts = tessellated_shape.geometry.verts
                    vertsList = [verts[i:i+3] for i in range(0, len(verts), 3)]

                    faces = tessellated_shape.geometry.faces
                    facesList = [faces[i:i+3] for i in range(0, len(faces), 3)]

                    pointlist = self.ifcModel.createIfcCartesianPointList3D(
                        vertsList)
                    shape = self.ifcModel.createIfcTriangulatedFaceSet(pointlist,
                        None, None, facesList, None)

                    body_representation = self.ifcModel.createIfcShapeRepresentation(
                        context, "Body", "Tessellation", [shape])
                    new_representation = self.ifcModel.createIfcProductDefinitionShape(
                        None, None, [body_representation])

                    representation = tuple(new_representation)

                except Exception as e:
                    print(str(e) + ': Unable to generate OBJ data for ' +
                          str(product))

    def remove_ownerhistory(self):
        for entity in self.ifcModel.by_type('IfcOwnerHistory'):
            self.ifcModel.remove(entity)

    def remove_geometry(self):
        removeTypes = ['IfcLocalPlacement', 'IfcRepresentationMap', 'IfcGeometricRepresentationContext', 'IfcGeometricRepresentationSubContext', 'IfcProductDefinitionShape',
                       'IfcMaterialDefinitionRepresentation', 'IfcShapeRepresentation', 'IfcRepresentationItem', 'IfcStyledRepresentation', 'IfcPresentationLayerAssignment', 'IfcTopologyRepresentation']
        for ifcType in removeTypes:
            # print(ifcType)
            # (lambda x: self.ifcModel.remove(x), self.ifcModel.by_type(ifcType))
            for entity in self.ifcModel.by_type(ifcType):
                self.ifcModel.remove(entity)

In [ ]:
# Specify the input IFC file path
ifc_file_path = r"/content/sample_data/7m900_tue_hello_wall_with_door_roundtrip.ifc"

# Get the directory path and base name of the input file
directory = os.path.dirname(ifc_file_path)
base_name = os.path.splitext(os.path.basename(ifc_file_path))[0]

# Convert IFC to JSON using IFC2JSON4
json_file_path_4 = os.path.join(directory, base_name + "_4.json")
with open(json_file_path_4, 'w') as outfile:
    json.dump(IFC2JSON4(ifc_file_path).spf2Json(), outfile, indent=2)

# Convert IFC to JSON using IFC2JSON5a
json_file_path_5a = os.path.join(directory, base_name + "_5a.json")
with open(json_file_path_5a, 'w') as outfile:
    json.dump(IFC2JSON5a(ifc_file_path).spf2Json(), outfile, indent=2)

## Validation Json file ##



In [ ]:
import json
from jsonschema import validate, ValidationError

# Specify the path to the JSON file to validate
jsonFilePath = "/content/sample_data/7m900_tue_hello_wall_with_door_roundtrip_4.json"

# Load the JSON schema from the file
with open("/content/sample_data/IFC4x2-entities.json", "r") as schemaFile:
    schema = json.load(schemaFile)

# Load the JSON instance from the file
with open(jsonFilePath, "r") as jsonFile:
    instance = json.load(jsonFile)

try:
    validate(instance=instance, schema=schema)
    print("The JSON instance is valid against the schema.")
except ValidationError as e:
    print("Validation failed. The JSON instance is not valid against the schema.")
    print("Validation error:", e)

    # Access error details
    for error in e.errors:
        print("Error in block:", ".".join(str(x) for x in error.path))
        print("Validation message:", error.message)



In [ ]:
import json
import pandas as pd


In [ ]:
df = pd.read_json('/content/sample_data/Duplex_A_20110907_optimized_roundtrip.json')


In [ ]:
df.columns

In [ ]:
df['data'].iloc[0]

In [ ]:
type(df)

In [ ]:
len(df)

In [ ]:

df.values

In [ ]:

df.dtypes

In [ ]:
def flatten_json(json_data, prefix=''):
    flattened_data = {}

    for key, value in json_data.items():
        new_key = prefix + '.' + key if prefix else key

        if isinstance(value, dict):
            flattened_data.update(flatten_json(value, prefix=new_key))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                item_key = f"{new_key}[{i}]"
                if isinstance(item, dict):
                    flattened_data.update(flatten_json(item, prefix=item_key))
                else:
                    flattened_data[item_key] = item
        else:
            flattened_data[new_key] = value

    return flattened_data


In [ ]:
def flatten_json1(json_data, columns_to_flatten=[], prefix=''):
    flattened_data = {}

    for key, value in json_data.items():
        new_key = prefix + '.' + key if prefix else key

        if key in columns_to_flatten:
            if isinstance(value, dict):
                flattened_data.update(flatten_json(value, columns_to_flatten, prefix=new_key))
            elif isinstance(value, list):
                for i, item in enumerate(value):
                    item_key = f"{new_key}[{i}]"
                    if isinstance(item, dict):
                        flattened_data.update(flatten_json(item, columns_to_flatten, prefix=item_key))
                    else:
                        flattened_data[item_key] = item
            else:
                flattened_data[new_key] = value
        else:
            flattened_data[new_key] = value

    return flattened_data


In [ ]:
def flatten_json2(json_data, columns_to_flatten=[], prefix=''):
    flattened_data = {}

    for key, value in json_data.items():
        new_key = prefix + '.' + key if prefix else key

        if key in columns_to_flatten:
            if isinstance(value, dict):
                flattened_data.update(flatten_json(value, columns_to_flatten, prefix=new_key))
            elif isinstance(value, list):
                for i, item in enumerate(value):
                    item_key = f"{new_key}[{i}]"
                    if isinstance(item, dict):
                        flattened_data.update(flatten_json(item, columns_to_flatten, prefix=item_key))
                    else:
                        flattened_data[item_key] = item
            else:
                flattened_data[new_key] = value
        else:
            flattened_data[new_key] = value

    return flattened_data


In [ ]:
flattened_data = flatten_json(df)

In [ ]:
df

In [ ]:
from pandas.io.json import json_normalize
data_df = json_normalize(df["data"])

In [ ]:
data_df


In [ ]:
columns_to_flatten = [col for col in data_df.columns if isinstance(data_df[col].iloc[0], list)]
columns_to_flatten

In [ ]:
# Flatten the JSON data
flattened_data = flatten_json(data_df, columns_to_flatten)

# Convert the flattened data to a DataFrame
flattened_df = pd.DataFrame.from_dict(flattened_data, orient='index', columns=['Flattened Values'])

# Display the flattened DataFrame
print(flattened_df)


In [ ]:
nan = data_df.isna().sum()
nan

In [ ]:
# Calculate the percentage of null values for each column
null_percentages = (data_df.isnull().sum() / len(data_df)) * 100
# Set a threshold for the maximum allowed null percentage
threshold = 50  # Adjust the threshold as needed

# Filter out columns that exceed the threshold
columns_to_remove = null_percentages[null_percentages > threshold].index
columns_to_remove
# Remove the columns from the DataFrame
data_df = data_df.drop(columns=columns_to_remove)


In [ ]:
data_df

In [ ]:
data = pd.json_normalize(df['data'], meta=df.columns.drop('data').tolist())
data

In [ ]:
list_columns = [col for col in data.columns if isinstance(data[col].iloc[0], list)]
list_columns

In [ ]:
data.shape[0]

In [ ]:
def flatten_json(json_data, prefix=''):
    flattened_data = {}

    for key, value in json_data.items():
        new_key = prefix + '.' + key if prefix else key

        if isinstance(value, dict):
            flattened_data.update(flatten_json(value, prefix=new_key))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                item_key = f"{new_key}[{i}]"
                if isinstance(item, dict):
                    flattened_data.update(flatten_json(item, prefix=item_key))
                else:
                    flattened_data[item_key] = item
        else:
            flattened_data[new_key] = value

    return flattened_data


# Read the JSON file
with open('/content/sample_data/Duplex_A_20110907_optimized.json', 'r') as file:
    json_data = json.load(file)

# Flatten the JSON data
flattened_data = flatten_json(json_data)

# Process or save the flattened data as needed
print(flattened_data)


In [ ]:
nan_counts = data[list_columns].isna().sum()
nan_counts

In [ ]:
data= data.drop(columns=list_columns)
data

In [ ]:
data

In [ ]:
# data2[data2.duplicated(keep=False)].shape[0]

In [ ]:
duplicated_columns = data.columns[data.columns.duplicated()]
if len(duplicated_columns) > 0:
    print("Duplicate columns detected:", duplicated_columns)
else:
    print("No duplicate columns found.")


In [ ]:
for key, value in data.items():
    print(key, type(value))


In [ ]:
data.info()

In [ ]:
# Iterates over the items of the DataFrame data1 and checks if each value is of type dictionary (dict). If a column's value is a dictionary, it prints a message indicating that the column contains dictionary values.
for column, value in data.items():
    if isinstance(value, dict):
        print(f"Column '{column}' contains dictionary values.")

In [ ]:
# columns which contain only null values in the data1 DataFrame.
null_columns = data.columns[data.isnull().all()]
null_columns

In [ ]:
# count the number of NaN values in each column of a DataFrame
nan_counts = data.isna().sum()
print(nan_counts)


In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport


In [ ]:
profile = ProfileReport(data)


In [ ]:
profile.to_widgets()  # Display the report in an interactive widget

In [ ]:
profile

In [ ]:
null_counts = data.isnull().sum()
null_counts

In [ ]:
len(data)

In [ ]:
# Counting the number of rows in the DataFrame
total_rows = len(data)

# Define the proportion of rows as the threshold
threshold_proportion = 0.8  # Example threshold proportion

# Calculate the threshold based on the proportion
threshold = threshold_proportion * total_rows
threshold

In [ ]:
# Get the columns with a high number of null values
columns_to_remove = null_counts[null_counts >= threshold].index
columns_to_remove

In [ ]:
data1 = data.drop(columns=columns_to_remove)

In [ ]:
data1

In [ ]:
# Check if a column contains lists
list_columns = [col for col in data.columns if data[col].apply(lambda x: isinstance(x, list)).any()]
list_columns

In [ ]:
profile = ProfileReport(data1)
profile

***BERT***

In [ ]:
!pip install transformers


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.utils.rnn import pad_sequence


In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Placeholder for NaN values in the text column
nan_placeholder = " "
def generate_bert_embeddings(text):
    # Replace NaN values with the placeholder
    text = text if pd.notnull(text) else nan_placeholder

    # Tokenize text
    tokens = tokenizer.encode(text, add_special_tokens=True)

    # Convert tokens to tensors
    input_ids = torch.tensor([tokens])

    # Generate BERT embeddings
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract the embeddings
    embeddings = outputs.last_hidden_state.squeeze(0).mean(0).numpy()
    return embeddings

# Define the maximum length for tokenization
max_length = 512  # Adjust this value as needed

# Iterate over the columns and generate BERT embeddings
for col in data1.columns:
    # Get the text data in the column
    text_data = data1[col]

    # Generate BERT embeddings for each text entry in the column
    embeddings = []
    for text in text_data:
        # Convert text entry to string
        text = str(text)

        # Replace NaN values with the placeholder
        text = text if pd.notnull(text) else nan_placeholder

        # Tokenize text
        tokens = tokenizer.encode(text, add_special_tokens=True)

        # Truncate the tokens to the maximum length
        tokens = tokens[:max_length]

        # Convert tokens to tensors
        input_ids = torch.tensor([tokens])

        # Generate BERT embeddings
        with torch.no_grad():
            outputs = model(input_ids)

        # Extract the embeddings
        embedding = outputs.last_hidden_state.squeeze(0).mean(0).numpy()

        # Append the embedding to the list
        embeddings.append(embedding.tolist())

    # Store the embeddings in a new column
    data1[col + '_embeddings'] = embeddings

# Display the updated DataFrame with BERT embeddings
print(data_df)


In [ ]:
!pip install pymongo

In [ ]:
import pymongo

In [ ]:
import pymongo
import urllib.parse

# Replace the connection details with your own
username = "admin"
password = "wF3bJ%Fg"
host = "18.170.83.79"
port = "8081"

# Escape special characters in the username and password
escaped_username = urllib.parse.quote_plus(username)
escaped_password = urllib.parse.quote_plus(password)

# Construct the connection string
connection_string = f"mongodb://{escaped_username}:{escaped_password}@{host}:{port}"

# Connect to MongoDB using the connection string
client = pymongo.MongoClient(connection_string)

In [ ]:
db = client["admin"]
collection = db[""]

In [ ]:
# import json
# import torch
# from transformers import BertTokenizer, BertModel
# from torch.nn.utils.rnn import pad_sequence

# # Load JSON data
# with open('/content/sample_data/7m900_tue_hello_wall_with_door.json', 'r') as f:
#     json_data = json.load(f)

# # Load BERT model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# # Initialize a list to store embeddings
# embeddings = []

# # Iterate over the JSON data
# for item in json_data:
#     text = item['text']

#     # Tokenize text
#     tokens = tokenizer.encode(text, add_special_tokens=True)

#     # Pad or truncate the sequences to the same length
#     max_length = 10  # Define the desired maximum length
#     padded_tokens = tokens[:max_length] + [0] * (max_length - len(tokens[:max_length]))

#     # Convert tokens to tensors
#     input_ids = torch.tensor(padded_tokens)

#     # Generate BERT embeddings
#     with torch.no_grad():
#         outputs = model(input_ids.unsqueeze(0))  # Add an extra dimension for batch processing

#     # Extract the embeddings
#     embedding = outputs.last_hidden_state.squeeze(0).mean(0).numpy()

#     # Store the embedding along with other relevant data
#     item['embedding'] = embedding.tolist()

#     embeddings.append(item)

# # Save the embeddings with updated JSON data
# with open('embeddings.json', 'w') as f:
#     json.dump(embeddings, f)